In [ ]:
from adversarial_lab.core import ALModel
from adversarial_lab.core.noise_generators import AdditiveNoiseGenerator
from adversarial_lab.attacks.whitebox import WhiteBoxMisclassification
from adversarial_lab.core.preprocessing import PreprocessingFromFunction
from adversarial_lab.utils.plotting import plot_images_and_noise

from PIL import Image
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions

In [ ]:
model = InceptionV3(weights='imagenet')
model = ALModel(model=model)

def preprocess(x, input):
    input = tf.cast(input, dtype=tf.float32)
    if len(input.shape) == 2:
        input = tf.expand_dims(input, axis=-1)
        input = tf.image.grayscale_to_rgb(input)

    elif len(input.shape) == 3 and input.shape[-1] == 1:
        input = tf.image.grayscale_to_rgb(input)

    input_tensor = tf.convert_to_tensor(input, dtype=tf.float32)
    resized_image = tf.image.resize(input_tensor, (299, 299))
    batch_image = tf.expand_dims(resized_image, axis=0)
    return preprocess_input(batch_image)

pp = PreprocessingFromFunction.create(preprocess)

In [ ]:
image = Image.open('data/panda.jpg')
image_array = np.array(image)

np.argmax(model.predict(pp.preprocess(image_array)), axis=1)

In [ ]:
from adversarial_lab.analytics import AdversarialAnalytics, ImageTracker, LossTracker, PredictionsTracker
from adversarial_lab.db.sqllite import SqlliteDB

db = SqlliteDB(db_path="whitebox.db")
analytics = AdversarialAnalytics(db, 
                                 [
                                     ImageTracker(), 
                                     LossTracker(track_batch=False), 
                                     PredictionsTracker(track_batch=False, strategy="first_topk")
                                 ], 
                                 "test", 
                                 True)

In [ ]:
from adversarial_lab.core.losses import CategoricalCrossEntropy
from adversarial_lab.core.losses.penalty import LpNorm

loss = CategoricalCrossEntropy(framework="tensorflow",
                               penalties=[LpNorm(framework="tensorflow", p=2)])

In [ ]:
attacker = WhiteBoxMisclassification(model, "cce", "adam", preprocessing=pp, analytics=analytics)

In [ ]:
noise = attacker.attack(image_array, target_class=924, epochs=100, strategy="random", verbose=3)

In [ ]:
plot_images_and_noise(pp.preprocess(image_array), noise, attacker.noise_generator)
